# Import Data 

Source Data: https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip '/content/drive/MyDrive/Dataset final project/archive.zip'

Archive:  /content/drive/MyDrive/Dataset final project/archive.zip
  inflating: PP_recipes.csv          
  inflating: PP_users.csv            
  inflating: RAW_interactions.csv    
  inflating: RAW_recipes.csv         
  inflating: ingr_map.pkl            
  inflating: interactions_test.csv   
  inflating: interactions_train.csv  
  inflating: interactions_validation.csv  


# Data Preprocessing


In [6]:
# import basic libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
sns.set()

#### Download Data


In [7]:
df_raw_recipe = pd.read_csv('/content/RAW_recipes.csv')
df_raw_recipe.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [8]:
df_PP_recipe = pd.read_csv('/content/PP_recipes.csv')
df_PP_recipe.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [9]:
df_PP_recipe = pd.merge(df_PP_recipe,df_raw_recipe)
df_PP_recipe.sort_values(by='i', inplace=True)
df_PP_recipe.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
46053,40893,0,"[40480, 1454, 16201, 2056, 955, 541, 11332, 82...","[[1424, 8876, 11007], [3484, 21453], [38966, 2...","[40480, 40482, 23667, 11007, 240, 21453, 240, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...",0,"[3384, 7979, 2127, 3502, 3217, 1257, 2778, 500...",white bean green chile pepper soup,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
108526,44394,1,"[40480, 34712, 22683, 11274, 5409, 29868, 40481]","[[5343, 535, 2044, 5409, 7087], [17869, 6020],...","[40480, 40482, 12172, 1281, 5409, 7087, 240, 6...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[912, 7557, 2499, 5382]",devilicious cookie cake delights,20,56824,2002-10-27,"['30-minutes-or-less', 'time-to-make', 'course...","[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]",5,"['blend together cake mix , oil and eggs', 'ad...",NaN,"[""devil's food cake mix"", 'vegetable oil', 'eg...",4
41248,85009,2,"[40480, 12187, 13995, 571, 14719, 40481]","[[1061, 494, 813, 2141], [31843], [30645, 4785...","[40480, 40482, 2572, 19472, 31757, 512, 823, 4...","[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...",2,"[4623, 6265, 1168, 6016, 3597, 3440, 7213, 169...",baked potato toppings,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13
28248,134728,3,"[40480, 27, 12, 263, 4017, 1494, 273, 14122, 4...","[[5867, 34056], [17918], [15022, 8361], [6953]...","[40480, 40482, 7087, 17918, 240, 6953, 488, 11...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[1304, 2683, 3217, 6270, 3532, 869, 7557, 3698...",kfc honey bbq strips,40,166019,2005-08-24,"['60-minutes-or-less', 'time-to-make', 'main-i...","[316.0, 4.0, 40.0, 37.0, 78.0, 4.0, 10.0]",10,"['mix flour , salt and pepper in bowl', 'set a...",these are so yummy and they do taste just like...,"['chicken tenders', 'flour', 'garlic powder', ...",12
177870,200236,4,"[40480, 16212, 11880, 556, 19811, 12772, 22075...","[[16212, 1535], [6953], [1430, 1301, 11332], [...","[40480, 40482, 2525, 10906, 485, 29848, 40478,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",2,"[4130, 6270, 3486, 7557, 5010, 3203, 2683, 125...",lamb stew with tomatoes chickpeas and spices,150,398869,2006-12-11,"['time-to-make', 'course', 'main-ingredient', ...","[606.5, 65.0, 12.0, 34.0, 65.0, 83.0, 7.0]",14,"['heat oven to 250', 'toss lamb with salt and ...",north african spices with a basic meat stew re...,"['lamb shoulder', 'salt', 'ground black pepper...",16


In [10]:
data,data2,data3 = '/content/interactions_test.csv', '/content/interactions_train.csv', '/content/interactions_validation.csv'

In [11]:
df = pd.read_csv(data)
df2 = pd.read_csv(data2)
df3 = pd.read_csv(data3)
df_train_rating = pd.concat([df,df2,df3]).drop_duplicates()
df_train_rating.head()

,user_id,recipe_id,date,rating,u,i
0,8937,44551,2005-12-23,4.0,2,173538
1,56680,126118,2006-10-07,4.0,16,177847
2,349752,219596,2008-04-12,0.0,26,89896
3,628951,82783,2007-11-13,2.0,45,172637
4,92816,435013,2013-07-31,3.0,52,177935


#### Create Utility Matrix

In [12]:
#Users-items utility matrix
from scipy.sparse import csr_matrix, vstack, hstack
from pandas.api.types import CategoricalDtype

item_c = CategoricalDtype(sorted(df_train_rating.i.unique()), ordered=True)
user_c = CategoricalDtype(sorted(df_train_rating.u.unique()), ordered=True)

col = df_train_rating.i.astype(item_c).cat.codes
row = df_train_rating.u.astype(user_c).cat.codes
interactions = csr_matrix((df_train_rating["rating"], (row, col)), \
                           shape=(user_c.categories.size,item_c.categories.size))

In [13]:
interactions

<25076x178265 sparse matrix of type '<class 'numpy.float64'>'
	with 718379 stored elements in Compressed Sparse Row format>

#### Create Dictionary containing Items and Users informations



In [14]:
def create_item_dict(df,id_col,name_col):
    item_dict = {}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

def create_user_dict(df,user_ids_col,u_col):
    user_id = list(df[user_ids_col])
    user_dict = {}
    u_index = list(df[u_col])
    for i in range(len(user_id)):
        user_dict[user_id[i]] = u_index[i]
    return user_dict
    

In [15]:
item_dict = create_item_dict(df_PP_recipe,'i','name')
#item_dict

In [16]:
user_dict = create_user_dict(df_train_rating,'user_id','u')
#user_dict

### Create Item's Features Map

In [ ]:
def convert_string_to_float(s):
  s = s.replace('[','').replace(']','')
  s = np.fromstring(s,dtype=float,sep=',')
  return s

#### Nutritions values


In [ ]:
str_nutritions = df_PP_recipe.nutrition.values
nutrition_labels = ['calories', 'total fat (PDV)', 'sugar (PDV)' , 'sodium (PDV)' , 'protein (PDV)' , 'saturated fat (PDV)', 'carbohydrates (PDV)']

In [ ]:
flt_nutritions = []
for i in str_nutritions:
  flt_nutritions.append(convert_string_to_float(i))

In [ ]:
nutritions_map = {'recipe_id':[]}
for i in nutrition_labels: 
  nutritions_map[i] = []

In [ ]:
ids = df_PP_recipe.i.values 
nutritions_map['recipe_id'] = ids 
calories = []
totalfat = []
sugar = []
na = []
protein = []
sat_fat = []
carb = []
for map in flt_nutritions: 
  calories.append(map[0])
  totalfat.append(map[1])
  sugar.append(map[2])
  na.append(map[3])
  protein.append(map[4])
  sat_fat.append(map[5])
  carb.append(map[6])

In [ ]:
nutritions_map['calories']=calories
nutritions_map['total fat (PDV)']=totalfat
nutritions_map['sugar (PDV)']=sugar
nutritions_map['sodium (PDV)']=na
nutritions_map['protein (PDV)']=protein
nutritions_map['saturated fat (PDV)']=sat_fat
nutritions_map['carbohydrates (PDV)']=carb

In [ ]:
df_nutritions = pd.DataFrame(data = nutritions_map)
df_nutritions.head()

,recipe_id,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,0,204.8,5.0,9.0,26.0,24.0,2.0,10.0
1,1,132.3,11.0,39.0,5.0,4.0,11.0,5.0
2,2,2786.2,342.0,134.0,290.0,161.0,301.0,42.0
3,3,316.0,4.0,40.0,37.0,78.0,4.0,10.0
4,4,606.5,65.0,12.0,34.0,65.0,83.0,7.0


#### Recipe's Requirements Informations

In [ ]:
minutes = df_PP_recipe.minutes.values
steps = df_PP_recipe.n_steps.values
ingredients = df_PP_recipe.n_ingredients.values

features = {'recipe_id':[],'minutes':[],'n_steps':[],'n_ingredient':[]}
features['recipe_id'] = ids
features['minutes'] = minutes
features['n_steps'] = steps
features['n_ingredient'] = ingredients

In [ ]:
df_embed = pd.DataFrame(data = features)
df_embed.head()

,recipe_id,minutes,n_steps,n_ingredient
0,0,495,4,9
1,1,20,5,4
2,2,10,3,13
3,3,40,10,12
4,4,150,14,16


In [ ]:
df_embed = pd.merge(df_embed,df_nutritions)
df_embed

,minutes,n_steps,n_ingredient,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),Healthy,Non-Veg dessert,Non-veg,Veg,Veg dessert,recipe_id
0,495,4,9,204.8,5.0,9.0,26.0,24.0,2.0,10.0,1,0,0,0,0,0
1,20,5,4,132.3,11.0,39.0,5.0,4.0,11.0,5.0,1,0,0,0,0,1
2,10,3,13,2786.2,342.0,134.0,290.0,161.0,301.0,42.0,0,0,1,0,0,2
3,40,10,12,316.0,4.0,40.0,37.0,78.0,4.0,10.0,0,0,1,0,0,3
4,150,14,16,606.5,65.0,12.0,34.0,65.0,83.0,7.0,0,0,0,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181210,45,11,16,839.2,63.0,2.0,50.0,77.0,55.0,25.0,0,0,1,0,0,178260
181211,13,13,4,16.3,0.0,0.0,0.0,0.0,0.0,1.0,0,0,1,0,0,178261
181212,70,9,5,505.9,44.0,19.0,18.0,63.0,39.0,8.0,0,0,0,1,0,178262
181213,375,23,10,518.1,29.0,138.0,26.0,34.0,35.0,24.0,0,0,1,0,0,178263


#### Define Food Type base on ingredients

In [ ]:
df_PP_recipe = pd.concat([df_PP_recipe,df_nutritions['calories']],axis=1)
df_PP_recipe['food types'] = np.nan
df_PP_recipe['food types'] = df_PP_recipe['food types'].astype('str')

for i in df_PP_recipe['ingredients'].index:
    if('eggs' not in df_PP_recipe['ingredients'][i]):
         if('ice-cream' in df_PP_recipe['ingredients'][i] or 'chocolate' in df_PP_recipe['ingredients'][i] or 'cookies' in df_PP_recipe['ingredients'][i]):
                df_PP_recipe['food types'][i]='Veg dessert'
    elif('eggs' in df_PP_recipe['ingredients'][i]):
        if('ice-cream' in df_PP_recipe['ingredients'][i] or 'chocolate' in df_PP_recipe['ingredients'][i] or 'cookies' in df_PP_recipe['ingredients'][i]):
                df_PP_recipe['food types'][i]='Non-Veg dessert'

for i in df_PP_recipe.index:
    if(df_PP_recipe['food types'][i]!='Veg dessert' and df_PP_recipe['food types'][i]!='Non-Veg dessert' and 20<df_PP_recipe['calories'][i]<300):
        df_PP_recipe['food types'][i]='Healthy'

for i in df_PP_recipe.index:
    if(df_PP_recipe['food types'][i]!='Veg dessert' and df_PP_recipe['food types'][i]!='Non-Veg dessert' and df_PP_recipe['food types'][i]!='Healthy'):
        if('chicken' in df_PP_recipe['ingredients'][i] or 'eggs' in df_PP_recipe['ingredients'][i] or'ham' in df_PP_recipe['ingredients'][i] or 'pepperoni' in df_PP_recipe['ingredients'][i] ):
            df_PP_recipe['food types'][i]='Non-veg'
for i in df_PP_recipe.index:
    if(df_PP_recipe['food types'][i]!='Veg dessert' and df_PP_recipe['food types'][i]!='Non-Veg dessert' and df_PP_recipe['food types'][i]!='Healthy' and df_PP_recipe['food types'][i]!='Non-veg'):
        df_PP_recipe['food types'][i]='Veg'

In [ ]:
df_PP_recipe['food types'].value_counts()

Healthy            79237
Veg                58766
Non-veg            29498
Veg dessert         7065
Non-Veg dessert     3699
Name: food types, dtype: int64

In [ ]:
df_PP_recipe['food types'].isnull().sum()

0

In [ ]:
types = pd.get_dummies(df_PP_recipe['food types'])

In [ ]:
df_embed = pd.concat([df_embed,types],axis=1)
df_embed.drop(columns=(['recipe_id']),axis=1,inplace=True)
df_embed.head()

,minutes,n_steps,n_ingredient,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),Healthy,Non-Veg dessert,Non-veg,Veg,Veg dessert
0,495,4,9,204.8,5.0,9.0,26.0,24.0,2.0,10.0,1,0,0,0,0
1,20,5,4,132.3,11.0,39.0,5.0,4.0,11.0,5.0,1,0,0,0,0
2,10,3,13,2786.2,342.0,134.0,290.0,161.0,301.0,42.0,0,0,1,0,0
3,40,10,12,316.0,4.0,40.0,37.0,78.0,4.0,10.0,0,0,1,0,0
4,150,14,16,606.5,65.0,12.0,34.0,65.0,83.0,7.0,0,0,0,1,0


In [ ]:
item_features = csr_matrix(df_embed.values)
item_features

<178265x15 sparse matrix of type '<class 'numpy.float64'>'
	with 1900823 stored elements in Compressed Sparse Row format>

### Users Features

In [ ]:
df_users = pd.read_csv('/content/PP_users.csv')
df_users.head()

,u,techniques,items,n_items,ratings,n_ratings
0,0,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...",31,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...",31
1,1,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...",39,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",39
2,2,"[13, 0, 0, 7, 5, 0, 1, 2, 1, 11, 0, 1, 0, 0, 1...","[168054, 87218, 35731, 1, 20475, 9039, 124834,...",27,"[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...",27
3,3,"[498, 13, 4, 218, 376, 3, 2, 33, 16, 591, 10, ...","[163193, 156352, 102888, 19914, 169438, 55772,...",1513,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...",1513
4,4,"[161, 1, 1, 86, 93, 0, 0, 11, 2, 141, 0, 16, 0...","[72857, 38652, 160427, 55772, 119999, 141777, ...",376,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...",376


In [ ]:
user_u = df_users.u.values
user_features_dict = {'u':user_u}
labels = ['Healthy','Veg dessert','Non-Veg dessert','Veg','Non-veg']
for i in labels: 
  user_features_dict[i] = []

In [ ]:
users_embed =[]
for u in user_u: 
  df_u = df_users.loc[df_users['u']==u]
  str_item_list = df_u['items'].values
  str_rating_list = df_u['ratings'].values
  item_list = convert_string_to_float(str_item_list[0])
  rating_list = convert_string_to_float(str_rating_list[0])
  type_item_list = []
  u_embed = []
  for i in item_list: 
      df = df_PP_recipe.loc[df_PP_recipe['i']==i]
      type_item_list.append(df['food types'].values[0])
  sum = np.sum(rating_list)

  for j in labels:
      sum_type_rating = 0 
      embed_value = 0
      if j not in type_item_list:
        u_embed.append(0)
      else:
        for index in range(len(type_item_list)):
          if j == type_item_list[index]:    
              sum_type_rating += rating_list[index]
        if sum ==0:
          embed_value -= 0.1
        else:
          embed_value = sum_type_rating/sum
        u_embed.append(embed_value)
  if np.sum(u_embed) != 1:
      u_embed = np.array(u_embed)
      fill_value =  - np.sum(u_embed)
      zeros = np.where(u_embed==0)[0]
      fill_value = fill_value / len(zeros)
      u_embed = np.where(u_embed==0,fill_value,u_embed)
  users_embed.append(u_embed)

In [ ]:
healthy = []
veg_dsrt = []
nonveg_dsrt = []
veg = []
nonveg  = []
for user in users_embed:
    healthy.append(user[0])
    veg_dsrt.append(user[1])
    nonveg_dsrt.append(user[2])
    veg.append(user[3])
    nonveg.append(user[4])
user_features_dict['Healthy'] = healthy
user_features_dict['Veg dessert'] = veg_dsrt
user_features_dict['Non-Veg dessert'] = nonveg_dsrt
user_features_dict['Veg'] = veg
user_features_dict['Non-veg'] = nonveg

In [ ]:
df_u_embed = pd.DataFrame(data= user_features_dict)
df_u_embed.drop(columns=(['u']),axis=1,inplace=True)
df_u_embed

,Healthy,Veg dessert,Non-Veg dessert,Veg,Non-veg
0,0.322148,0.000000,0.033557,0.389262,0.255034
1,0.378238,0.000000,0.000000,0.362694,0.259067
2,0.596491,0.000000,0.000000,0.192982,0.210526
3,0.481426,0.018643,0.015490,0.310350,0.174092
4,0.402424,0.034545,0.031515,0.330303,0.201212
...,...,...,...,...,...
25071,1.000000,0.000000,0.000000,0.000000,0.000000
25072,0.000000,0.000000,0.000000,0.500000,0.500000
25073,0.000000,0.000000,0.000000,1.000000,0.000000
25074,0.000000,0.000000,0.000000,0.500000,0.500000


In [ ]:
user_features = csr_matrix(df_u_embed)
user_features

<25076x5 sparse matrix of type '<class 'numpy.float64'>'
	with 74645 stored elements in Compressed Sparse Row format>

# Recommender System 


### LightFM Implementation


In [ ]:
!pip install -qq lightfm

In [ ]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import *

In [ ]:
# Split interactions to train and test sets 
train, test = random_train_test_split(interactions,test_percentage=0.1,random_state=42)

##### Define LightFM model

In [ ]:
hybrid = LightFM(no_components=32,random_state=42,loss='warp',item_alpha=1e-06,user_alpha=1e-06)
hybrid.fit(train,user_features,item_features,epochs=10,num_threads=4,verbose=True)

Epoch: 100%|██████████| 10/10 [00:41<00:00,  4.11s/it]


##### Evaluation: AUC score


In [ ]:
hybrid_train_auc = auc_score(hybrid,train,item_features=item_features,user_features=user_features,num_threads=4)
hybrid_test_auc = auc_score(hybrid,test,train_interactions=train,item_features=item_features,user_features=user_features,num_threads=4)

In [ ]:
print('Hybrid model train AUC score: %.5f' %hybrid_train_auc.mean())
print('Hybrid model test AUC score: %.5f' %hybrid_test_auc.mean())

Hybrid model train AUC score: 0.53597
Hybrid model test AUC score: 0.53274


### Recommend Definitions
[Link to code](https://github.com/aayushmnit/cookbook/blob/master/recsys.py)

In [20]:
def sample_recommendation_user(model, interactions, user_id, 
                               item_dict,threshold = 3,nrec_items = 0, show = True):
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_ids = user_x,item_ids=np.arange(n_users),item_features=item_features,user_features=user_features))
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    user_rated_items = interactions.getrow(user_x).nonzero()
    thr = np.where(interactions.getrow(user_x)[user_rated_items] > threshold)[1]
    user_rated_items = user_rated_items[0][thr]
    known_items = list(pd.Series([user_rated_items]).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items[0]]
    return_score_list = scores[0:nrec_items]
    known_names = []
    for i in known_items[0]:
      known_names.append(item_dict[i])
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Likes:")
        counter = 1
        for i in known_names:
            print(str(counter) + '- ' + i)
            counter+= 1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list


def recommendation(model, interactions, user_id, item_dict,threshold = 0,nrec_items = 0):
  rec_items_ids = sample_recommendation_user(model, interactions, user_id,item_dict,threshold,nrec_items, show = False)
  rec_items = []
  output = ''
  for i in rec_items_ids:
    rec_items.append(item_dict[i])
  output += 'Recipe you might like: \n'
  counter = 1
  for i in rec_items:
    output += str(counter) + '- ' + i + '\n'
    counter+=1
  return output



In [22]:
pip install -qq gradio

     |████████████████████████████████| 1.1MB 4.0MB/s 
     |████████████████████████████████| 1.9MB 39.8MB/s 
     |████████████████████████████████| 215kB 45.3MB/s 
     |████████████████████████████████| 962kB 42.1MB/s 
     |████████████████████████████████| 3.2MB 41.5MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 


In [23]:
name_dict = {}
import gradio as gr
import random

def username(name,name_dict):
  from random import randint
  new_i = len(user_dict)
  new_id = randint(0,1e7)
  while new_id in user_dict:
    new_id = randint(0,1e7)
  user_dict[new_id]=new_i
  name_dict[name] = new_id

  return name, new_id
def generate_id(name):
    new, user_id = username(name,name_dict)
    out = 'Welcome to FuKaKukku!\n'
    out += '------------------------------\n'
    out += 'Your new username: {}. Id: {} \n'.format(name,user_id)
    out += '------------------------------\n'
    out += 'Please sign in to join FukkaKukku!'
    return out
def recommend(are_you_new,name,veg,healthy,dsrt):
  if are_you_new: 
    if name in name_dict:
      return 'Username already existed!'
    else:
      out = generate_id(name)
      global interactions
      global user_features
      new_interact = csr_matrix(np.zeros((1,interactions.shape[1])))
      interactions = csr_matrix(vstack([new_interact,interactions]))
      new_u_features = np.zeros((user_features.shape[1]))
      if veg == 'Yes':
        new_u_features[3] += 0.2
        new_u_features[4] -= 0.2
        new_u_features[2] -= 0.2
        new_u_features[1] += 0.2
      else:
        new_u_features[3] += 0.2
        new_u_features[4] += 0.2
        new_u_features[2] += 0.2
        new_u_features[1] += 0.2
      if healthy == 'Yes':
        new_u_features[0] += 0.2
      else:
        new_u_features[0] -= 0.2
      if dsrt == 'Yes':
        if veg =='Yes':
          new_u_features[2] -= 0.2
          new_u_features[1] += 0.2
        else:
          new_u_features[2] += 0.2
          new_u_features[1] += 0.2
      else:
        new_u_features[2] -= 0.5
        new_u_features[1] -= 0.5
        sum = np.sum(new_u_features)
        for i in range(len(new_u_features)):
          new_u_features[i] = new_u_features[i]/sum
      new_u_features = csr_matrix(new_u_features)
      user_features = csr_matrix(vstack([new_u_features,user_features]))
      return out
  else:
    if name not in name_dict: 
      warning = 'You have not registered yet. \n'
      warning += '------------------------------\n'
      warning += 'Please check the New to FuKKaKukku checkbox to register.'
      return warning
    else:  
      user_id = name_dict[name]
      rec = recommendation(hybrid,interactions,user_id,item_dict,threshold = 3)
      greet = 'Hello {}! Welcome to FuKaKukku! \n'.format(name)
      greet += rec
      return greet

##  Demo UI using Gradio
<br>

#### How to use:
- For new users:
  - Tick *New to FuKaKukku* checkbox and type your new user name and check the other checkboxes to describe yourself.
  - Submit to register
  - Untick *New to FuKaKukku* and Re-submit
  - The Recommender System will recommend items you might like.
- For registered users:
  - Type your username and ignore the other boxes since they only serve the purpose of create an initial user embedding.
  - Submit and the Recommender System will do the rest.
<br>
 
 ##### Of course this demo is solely for demo purpose, as the name suggested. We can not rate items. But we can input data using csv file and the functions under this interface.



In [24]:
iface = gr.Interface(
  fn=recommend, 
  inputs=[gr.inputs.Checkbox(label='New to FukkaKukku?'),
          gr.inputs.Textbox(label='Username:'),
          gr.inputs.Radio(['Yes','No'], label='Are you a vegetarian:'),
          gr.inputs.Radio(['Yes','No'], label='Do you want a Healthy dish:'),
          gr.inputs.Radio(['Yes','No'], label='Do you want a main dish:')
          ], 
  outputs=[gr.outputs.Textbox(label='Welcome')],
  title='Welcome to FuKaKukku'
  )
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://25441.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://25441.gradio.app')

#### New interactions data inputs Section


In [26]:
df_new = pd.read_csv('/content/drive/MyDrive/Dataset final project/Demo Data.csv')
df_new

,user_name,recipe_id,i,rating,Note
0,Mai,186383,105140,4.0,Dessert
1,Mai,288683,38144,3.5,Dessert
2,Mai,60244,74743,0.0,Dessert
3,Mai,281968,139097,2.5,Dessert
4,Mai,293330,54812,3.0,Dessert
...,...,...,...,...,...
148,An,450716,92941,4.0,Veg dessert
149,An,78538,27487,5.0,Healthy
150,An,88906,101591,2.0,Healthy
151,An,188204,156487,3.0,Healthy


##### Visualize Demo Data:
Link to demo data: [click here](https://docs.google.com/spreadsheets/d/ohd2GuRR8L2ebJfp2BnZj9aILw40IRqQvJB-iD6acA/edit#gid=20253765)

In [25]:
def user_update(model,interactions,user_features,new_df,name_col,i_col,rate_col,user_dict):
  '''
  This function updates rating values of registered users and re-compute user embedding
  Remember to register all the name in the data csv file before call this function
  '''
  username = new_df[name_col].values
  users = []
  for i in username:
    if len(users)==0:
      users.append(i)
    else:
      if i not in users:
        users.append(i)
        

  for n in users:
    user_id = name_dict[str(n)]
    u = user_dict[user_id]
    u_rating = np.zeros(shape=(interactions.shape[1]))
    df_user = new_df.loc[new_df[name_col]==n]
    item_i = df_user[i_col].values
    rating = df_user[rate_col].values
    for i in range(len(item_i)):
      u_rating[item_i[i]] = rating[i]
    interactions[u,:] = csr_matrix(u_rating)
    
    type_item_list = []
    u_embed = []
    for i in item_i: 
      df = df_PP_recipe.loc[df_PP_recipe['i']==i]
      type_item_list.append(df['food types'].values[0])
    sum_rating = np.sum(rating)
    for j in labels:
      sum_type_rating = 0 
      embed_value = 0
      if j not in type_item_list:
        u_embed.append(0)
      else:
        for index in range(len(type_item_list)):
          if j == type_item_list[index]:    
              sum_type_rating += rating[index]
        if sum ==0:
          embed_value -= 0.
        else:
          embed_value = sum_type_rating/sum
        u_embed.append(embed_value)
    if np.sum(u_embed) != 1:
        u_embed = np.array(u_embed)
        fill_value =  - np.sum(u_embed)
        zeros = np.where(u_embed==0)[0]
        fill_value = fill_value / len(zeros)
        u_embed = np.where(u_embed==0,fill_value,u_embed)
    user_features[u,:] = csr_matrix(u_embed)


  model.fit(interactions,item_features=item_features,user_features=user_features,epochs=5,num_threads=4,verbose = True)   

  print('Updated')
  return interactions, user_features, model

In [ ]:
user_update(hybrid,interactions,user_features,df_new,'user_name','i','rating',user_dict)

###### Reset interactions, user_dict, name_dict in case something gone wrong


In [ ]:
interactions = csr_matrix((df_train_rating["rating"], (row, col)), \
                           shape=(user_c.categories.size,item_c.categories.size))
user_dict = create_user_dict(df_train_rating,'user_id','u')
name_dict = {}
user_features = csr_matrix(df_u_embed)